<a href="https://colab.research.google.com/github/haosulab/SAPIEN-tutorial/blob/master/basics/5_contact.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Note: Some core features of SAPIEN are not available on Colab, including the interactive viewer and ray-tracing functionalities. You need to run SAPIEN locally for full features. You can also find the latest SAPIEN tutorial at [SAPIEN's documentation](https://sapien.ucsd.edu/docs/latest/index.html).

# Basics Tutorial 5: Contact

Contact information is useful to check whether two rigid bodies collide or whether an object is grasped by a gripper. The example shows how to check the contact between two actors (one box supported by another box).

In this tutorial, you will learn the following:

- Get contact information from `Contact`

## Preparation

In [ ]:
%pip install sapien

import sapien.core as sapien
import numpy as np

## Full Script

A full working script is provided as follows:

In [ ]:
engine = sapien.Engine()
scene = engine.create_scene()
dt = 1 / 100.0
scene.set_timestep(dt)

# ---------------------------------------------------------------------------- #
# Add two boxes
# ---------------------------------------------------------------------------- #
actor_builder = scene.create_actor_builder()
actor_builder.add_box_collision(half_size=[0.5, 0.5, 0.5])
# actor_builder.add_box_visual(half_size=[0.5, 0.5, 0.5], color=[1, 0, 0])
box1 = actor_builder.build_kinematic(name='box1')
box1.set_pose(sapien.Pose(p=[0, 0, 1.0]))
print('Mass of box1:', box1.mass)

actor_builder = scene.create_actor_builder()
actor_builder.add_box_collision(half_size=[0.25, 0.25, 0.25])
# actor_builder.add_box_visual(half_size=[0.25, 0.25, 0.25], color=[0, 1, 0])
box2 = actor_builder.build(name='box2')
box2.set_pose(sapien.Pose(p=[0, 0, 1.75]))
print('Mass of box2:', box2.mass)

# ---------------------------------------------------------------------------- #
# Check contacts
# ---------------------------------------------------------------------------- #
scene.step()
contacts = scene.get_contacts()
support_force = 0
for contact in contacts:
    print(contact)
    for point in contact.points:
        print('Impulse (F * dt) on the first actor:', point.impulse)
        print('Normal (same direction as impulse):', point.normal)
        print('Contact position (in the world frame):', point.position)
        print('Minimum distance between two shapes:', point.separation)
        if contact.actor0.name == 'box2':
            support_force += point.impulse[2] / dt
        elif contact.actor0.name == 'box1':
            support_force -= point.impulse[2] / dt
        else:
            raise RuntimeError('Impossible case in this example.')
# Sanity check: the support force should balance the gravity
np.testing.assert_allclose(support_force, 9.81 * box2.mass, rtol=1e-3)

You can call `get_contacts` to fetch all contacts after the current simulation step. It returns a list of `Contact`. `contact.actor0` and `contact.actor1` refer to two actors involved in the contact. `contact.points` contains a list of `ContactPoint`.

For each contact point,

- `impulse`: the impulse applied on the first actor.
- `normal`: the direction of impulse.
- `position`: the point of application in the world frame.
- `seperation`: minimum distance between two shapes involved in the contact.

> Note: `Contact` in SAPIEN does not mean that two actors are contacting each other. It will be generated when the contact is about to start or end, and, of course, when the contact is happening.